In [ ]:
!pip install transformers==4.36.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [ ]:
from transformers import pipeline

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn


In [ ]:
from transformers import Conversation

In [ ]:
chatbot = pipeline("conversational",
                   model="facebook/blenderbot-400M-distill")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
!ngrok config add-authtoken "paste your ngrok authtoken code"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from transformers import BlenderbotTokenizer

tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-71' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [ ]:
def count_tokens(conversation):
    # Join all messages into a single text string
    text = " ".join([msg["content"] for msg in conversation])
    return len(tokenizer.encode(text, add_special_tokens=False))


In [ ]:
def limit_conversation_tokens(conversation, max_tokens=128):
    print(conversation.messages)
    while count_tokens(conversation) > max_tokens:
        print(conversation.messages[0:2])
        del conversation.messages[0:2]
        print(conversation.messages)



In [ ]:
conversation = Conversation()
def chatbot_model(user_input):
    print("hi, i am here")
    conversation.add_message({
        "role":"user",
        "content" : user_input
    })


    limit_conversation_tokens(conversation)
    response = chatbot(conversation)
    return response.generated_responses[-1]


In [ ]:
api = FastAPI()

class UserInput(BaseModel):
  text:str

In [ ]:
@api.post("/chat")
async def chat(user_input:UserInput):
  print(user_input.text)
  user_input = user_input.text
  if user_input.lower() in ["bye", "quit", "exit", "goodbye", "stop", "end", "cancel"]:
    return {"response" :"goodbye"}
  reply = chatbot_model(user_input)
  print(reply)
  return {"response" : reply}

In [ ]:
from fastapi.middleware.cors import CORSMiddleware

api.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500", "http://localhost:5000"],  # add all your frontend origins here
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [ ]:

nest_asyncio.apply()
ngrok.kill()
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run API
uvicorn.run(api, port=8000)

Public URL: NgrokTunnel: "https://242851c81df5.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [883]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.179.246.205:0 - "OPTIONS /chat HTTP/1.1" 200 OK
Hello, how are you?
hi, i am here
[{'role': 'user', 'content': 'Hello, how are you?'}]
 I'm doing well, thank you. How are you this fine evening? Do you have any plans?
INFO:     103.179.246.205:0 - "POST /chat HTTP/1.1" 200 OK
No I am free
hi, i am here
[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': " I'm doing well, thank you. How are you this fine evening? Do you have any plans?"}, {'role': 'user', 'content': 'No I am free'}]
 What do you do for a living? I work at a grocery store as a cashier.
INFO:     103.179.246.205:0 - "POST /chat HTTP/1.1" 200 OK
haha that's funny I know you don't work
hi, i am here
[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': " I'm doing well, thank you. How are you this fine evening? Do you have any plans?"}, {'role': 'user', 'content': 'No I am free'}, {'role': 'assistant', 'content': ' What do you do for a living? I 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [883]
